# Part 3.1

Сейчас мы с вами поиграем с RNN, которая решает задачу генерации текста.

Ваша задача - поиграть с параметрами, и попробовать сделать так, чтобы ошибка модели была по крайней мере не больше 0.5

In [ ]:
!pip install livelossplot

## Импорты и функции

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from livelossplot import PlotLossesKeras
import random
import sys

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Загрузка и предобработка датасета

In [ ]:
with open('garri-potter.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))

chars = sorted(list(set(text))) # getting all unique chars
print('total chars: ', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Построение модели

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars))))            # Это можно подбирать
model.add(Dropout(0.2))                                           # Это можно подбирать
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.002)                          # Это можно подбирать
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.0001)          # Это можно подбирать

callbacks = [reduce_lr, PlotLossesKeras()]

## Обучение

In [ ]:
model.fit(x, y, batch_size=128, epochs=10, callbacks=callbacks)   # Это можно подбирать

## Генерация текста

In [ ]:
def generate_sentence(sentence, diversity = 1.2):
  generated = ''
  generated += sentence
  sentence = sentence[-maxlen:]
  sys.stdout.write(generated)

  for i in range(400):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.

      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]

      generated += next_char
      sentence = sentence[1:] + next_char

      sys.stdout.write(next_char)
      sys.stdout.flush()
  return generated

In [ ]:
_ = generate_sentence('гарри, рон, и гермиона зашли в комнату. там сидел драко и ехидно смотрел в стену. троица не растерялась и жестко отчислилась.')